In [1]:
import numpy as np
import matplotlib as plt

In [3]:
#Functions for x and y arrays

def call_indicators(key):
    '''This function calls the Technical Indicators class from the Alpha Vantage package.
    Input: 
    key: Alpha Vantage API Key
    '''
    from alpha_vantage.techindicators import TechIndicators
    return TechIndicators(key, output_format='pandas')

def call_timeseries(key):
    '''This function calls the Time Series class from the Alpha Vantage package.
    Input: 
    key: Alpha Vantage API Key
    '''
    from alpha_vantage.timeseries import TimeSeries
    return TimeSeries(key, output_format='pandas')

def array_xs(symbol, key):
    '''Returns a JSON Dictionary containing the last data points for a specified indicator on a specific stock.
    Numpy must be imported before use.
    Inputs:
    - indicator: technical indicator, chosen from **LIST HERE**
    - symbol of the stock chosen, eg. Microsoft
    - key: Alpha Vantage API Key
    Returns: numpy array'''
    
    #Call the Technical Indicators class from Alpha Vantage
    techin = call_indicators(key)
    
    mom_data, meta_mom = techin.get_mom(symbol=symbol, interval='daily')
    large_MOM = np.array(mom_data['MOM'])
    small_MOM = large_MOM[-1:-200:-1]
    small_MOM = small_MOM.reshape(small_MOM.shape[0], -1)
    #print(small_MOM.shape)
    
    sma_data, meta_sma = techin.get_sma(symbol, interval='daily', time_period=200, series_type='close')
    large_SMA = np.array(sma_data['SMA'])
    small_SMA = large_SMA[-1:-200:-1]
    small_SMA = small_SMA.reshape(small_SMA.shape[0], -1)
    #print(small_SMA.shape)
    
    #x = np.concatenate((small_MOM, small_SMA), axis=0)
    x = np.concatenate((small_MOM, small_SMA), axis=1)
    
    rsi_data , meta_rsi = techin.get_rsi(symbol, interval='daily', time_period=200, series_type='close')
    large_rsi = np.array(rsi_data['RSI'])
    small_rsi = large_rsi[-1:-200:-1]
    small_rsi = small_rsi.reshape(small_rsi.shape[0], -1)
    #print(small_rsi.shape)
    
    #x = np.concatenate((x, small_rsi), axis=0)
    x = np.concatenate((x, small_rsi), axis=1)
    return x

def label_ys(symbol, key):
    '''Returns a vector y with enteries labelled 1 or 0 based on whether the percentage increase has been
    met over a specific time period.
    Numpy must be imported before use.'''
    
    #Call the Time Series class from Alpha Vantage
    ts = call_timeseries(key)
    
    data, meta_data = ts.get_daily(symbol, outputsize='full')
    
    data_array = np.array(data['4. close'])
    smaller_array = data_array[-1:-207:-1]
    
    y = np.zeros(199)
    for i in range(7, 206):
        if smaller_array[i-7] >= 1.05*smaller_array[i]:
            y[i-7] = 1
    
    return y

In [4]:
#Initionalize arrays

x = array_xs(symbol='MSFT',key=api_key)
x = x.reshape(x.shape[0], -1).T
#print(x.shape)

y = label_ys(symbol='MSFT',key=api_key)
y = y.reshape(-1, y.shape[0])
#print(y.shape)

In [5]:
# Define activation functions
def sigmoid(Z):
    A = 1 / (1 + np.exp(-Z))
    return A, Z

def tanh(Z):
    A = np.tanh(Z)
    return A, Z

def relu(Z):
    A = np.maximum(0, Z)
    return A, Z

def leaky_relu(Z):
    A = np.maximum(0.1*Z, Z)
    return A, Z

In [6]:
#Initialize parameters to random values between 0 and 1
def initialize_parameters(layers_dims):
    np.random.seed(1)
    parameters = {}
    L = len(layers_dims)
    
    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layers_dims[l], layers_dims[l - 1]) * 0.01
        parameters['b' + str(l)] = np.zeros((layers_dims[l], 1))
        
        assert parameters['W' + str(l)].shape == (layers_dims[l], layers_dims[l-1])
        assert parameters['b' + str(l)].shape == (layers_dims[l], 1)
        
    return parameters

In [7]:
#Define helper functions
def linear_forward(A_prev, W, b):
    Z = np.dot(W, A_prev) + b
    cache = (A_prev, W, b)

    return Z, cache

def linear_activation_forward(A_prev, W, b, activation_fn):
    assert activation_fn == 'sigmoid' or activation_fn == 'tanh' or activation_fn == 'relu'  
    
    if activation_fn == 'sigmoid':
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
        
    elif activation_fn == 'tanh':
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = tanh(Z)
        
    elif activation_fn == 'relu':
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
        
    assert A.shape == (W.shape[0], A_prev.shape[1]) 
    
    cache = (linear_cache, activation_cache)
    return A, cache

def L_model_forward(X, parameters, hidden_layers_activation_fn='relu'):
    A = X
    caches = []
    L = len(parameters)//2
    
    for l in range(1, L):
        A_prev = A
        A, cache = linear_activation_forward(A_prev, parameters['W'+str(l)], parameters['b'+str(l)], activation_fn=hidden_layers_activation_fn)
        caches.append(cache)
        
    AL, cache = linear_activation_forward(A, parameters['W'+str(L)], parameters['b'+str(L)], activation_fn='sigmoid')
    caches.append(cache)
    
    assert AL.shape == (1, X.shape[1])
    return AL, caches

NameError: name 'AL' is not defined

In [18]:
# Calculate the cross-entropy cost
def compute_cost(AL, y):
    #print(y.shape)
    #print(AL.shape)
    #print(np.log(AL))
    m = y.shape[1]
    cost = - (1 / m) * np.sum(np.multiply(y, np.log(AL)) + np.multiply(1 - y, np.log(1 - AL)))
    return cost

In [10]:
# Define gradient functions
def sigmoid_gradient(dA, Z):
    A, Z = sigmoid(Z)
    dZ = dA * A * (1 - A)
    return dZ

def tanh_gradient(dA, Z):
    A, Z = tanh(Z)
    dZ = dA * (1 - np.square(A))
    return dZ

def relu_gradient(dA, Z):
    A, Z = relu(Z)
    dZ = np. multiply(dA, np.int64(A > 0))
    return dZ

In [11]:
# Define helper functions to be used in the L-Model backwards propagation
def linear_backword(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[0]
    
    dW = (1 - m) * np.dot(dZ, A_prev.T)
    db = (1 - m) * np.sum(dZ, axis=1, keepdims=True)
    
    dA_prev = np.dot(W.T, dZ)
    
    assert dA_prev.shape == A_prev.shape
    assert dW.shape == W.shape
    assert db.shape == b.shape
    
    return dA_prev, dW, db

def linear_activation_backward(dA, cache, activation_fn):
    linear_cache, activation_cache = cache
    
    if activation_fn == 'sigmoid':
        dZ = sigmoid_gradient(dA, activation_cache)
        dA_prev, dW, db = linear_backword(dZ, linear_cache)
        
    elif activation_fn == 'tanh':
        dZ = tanh_gradient(dA, activation_cache)
        dA_prev, dW, db = linear_backword(dZ, linear_cache)
        
    elif activation_fn == 'relu':
        dZ = relu_gradient(dA, activation_cache)
        dA_prev, dW, db = linear_backword(dZ, linear_cache)
        
    return dA_prev, dW, db

def L_model_backwards(AL, y, caches, hidden_layers_activation_fn='relu'):
    y = y.reshape(AL.shape)
    L = len(caches)
    grads = {}
    
    dAL = np.divide(AL - y, np.multiply(AL, 1 - AL))
    
    grads['dA' + str(L-1)], grads['dW' + str(L)], grads['db' + str(L)] = linear_activation_backward(dAL, caches[L-1], 'sigmoid')
    
    for l in range(L-1, 0, -1):
        current_cache = caches[l-1]
        grads['dA' + str(l-1)], grads['dW' +str(l)], grads['db' + str(l)] = linear_activation_backward(grads['dA'+str(l)], current_cache, activation_fn=hidden_layers_activation_fn)
        
    return grads

In [12]:
def update_parameters(parameters, grads, learning_rate):
    L = len(parameters) // 2
    
    for l in range(1, L + 1):
        parameters['W' + str(l)] = parameters['W' + str(l)] - learning_rate * grads['dW' + str(l)]
        parameters['b' + str(l)] = parameters['b' + str(l)] - learning_rate * grads['db' + str(l)]
        
    return parameters

In [13]:
# Define L-layer model using all the helper functions built
def L_layer_model(X, y, layers_dims, learning_rate=0.01, num_iterations=3000, print_cost=True, hidden_layers_activation_fn='relu'):
    np.random.seed(1)
    
    #initialize parameters
    parameters = initialize_parameters(layers_dims)
    
    #initialize cost list
    cost_list = []
    
    #iterate over number of iterations
    for i in range(num_iterations):
        #iterate over L-layers to get the final output and the cache
        AL, caches = L_model_forward(X, parameters, hidden_layers_activation_fn)
        
        #compute cost to plot it
        cost = compute_cost(AL, y)
        
        #iterate over L-layers to get the gradients
        grads = L_model_backwards(AL, y, caches, hidden_layers_activation_fn)
        
        #update parameters
        parameters = update_parameters(parameters, grads, learning_rate)
        
        #append each 100th cost to the cost list
        if (i + 1) % 100 == 0 and print_cost:
            print(f'The cost after {i + 1} iterations is: {cost:.4f}')
            
        if (i + 1) % 100 == 0:
            cost_list.append(cost)
            
    #plot the cost curve
    plt.figure(figsize=(10,6))
    plt.plot(cost_list)
    plt.xlabel('Iterations (per hundreds)')
    plt.ylabel('Loss')
    plt.title(f'The loss curve for the learning rate = {learning_rate}')
    
def accuracy(X, parameters, y, activation_fn='relu'):
    probs, caches = L_model_forward(X, parameters, activation_fn)
    labels = (probs >= 0.5) * 1
    accuracy = np.mean(labels == y) * 100
    return f'The accuracy rate is {accuracy:.2f}%'

In [19]:
#Set the layers_dims
layers_dims = [x.shape[0], 5, 5, 1]
#NN with relu as the hidden activation function
parameters_relu = L_layer_model(x, y, layers_dims, learning_rate=0.01, num_iterations=3000, print_cost=True, hidden_layers_activation_fn='relu')

[[-0.6932047  -0.6932088  -0.69320326 -0.6932116  -0.69318946 -0.69320768
  -0.69319422 -0.69319062 -0.6931962  -0.69318579 -0.69317853 -0.69317456
  -0.69317561 -0.6931682  -0.69317614 -0.69317601 -0.69316775 -0.69316749
  -0.69316753 -0.69316657 -0.69316541 -0.69316507 -0.69316516 -0.69316528
  -0.69316497 -0.69316458 -0.69316481 -0.69316374 -0.69316467 -0.69316431
  -0.69316444 -0.69316465 -0.69316551 -0.69316655 -0.69316558 -0.69316632
  -0.69316699 -0.69316744 -0.69316674 -0.69316645 -0.6931666  -0.69316613
  -0.69316576 -0.69316595 -0.69316658 -0.69316655 -0.6931661  -0.69316611
  -0.69316587 -0.69316622 -0.69316669 -0.6931661  -0.69316613 -0.69316569
  -0.6931662  -0.69316654 -0.69316647 -0.69316645 -0.69316649 -0.69316615
  -0.69316611 -0.6931664  -0.69316674 -0.69316666 -0.69316643 -0.69316622
  -0.69316627 -0.69316646 -0.69316636 -0.69316627 -0.69316587 -0.69316611
  -0.69316618 -0.69316627 -0.69316605 -0.69316567 -0.69316573 -0.69316542
  -0.69316579 -0.69316539 -0.69316524 

C:\Users\Jessica\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log
  import sys
C:\Users\Jessica\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in multiply
  import sys
C:\Users\Jessica\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\Jessica\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in true_divide
C:\Users\Jessica\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in multiply
  after removing the cwd from sys.path.
C:\Users\Jessica\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in greater
  


[[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan]]
[[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan 

  nan]]
[[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan]]
[[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan 

  nan]]
[[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan]]
[[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan 

...

  nan]]
[[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan]]
[[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan 

[[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan]]
[[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan 

[[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan]]
[[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan 

  nan]]
[[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan]]
[[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan 

AttributeError: module 'matplotlib' has no attribute 'figure'